In [ ]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

from ibge import *
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Initialize DuckDB connection
conn = duckdb.connect("plants.db")

## SIBBR
### Converto para parquet

In [10]:
local_file = "./sibbr_parquet"
os.makedirs(local_file, exist_ok=True)

for p in pathlib.Path("./sibbr").rglob("data*.csv"):
    print(p)
    df = pd.read_csv(p)

    for column in df.columns:
        if df[column].dtype == "object":
            df[column] = df[column].astype(str)

    filename = os.path.join(local_file, "%s.parquet" % p.stem)
    df.to_parquet(filename)

sibbr/data_part3.csv


/tmp/ipykernel_2252338/1603162614.py:7: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p)


sibbr/data_part2.csv
sibbr/data.csv


/tmp/ipykernel_2252338/1603162614.py:7: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p)


sibbr/data_part4.csv


In [11]:
!ls ./sibbr_parquet

data.parquet  data_part2.parquet  data_part3.parquet  data_part4.parquet


## Sibbr
### Adiciono os registros no duckdb (via parquet)

In [12]:
conn.execute("DROP TABLE IF EXISTS sibbr")
conn.execute(f"""
        CREATE TABLE sibbr AS
        SELECT * FROM read_parquet('{local_file}', union_by_name=True)
    """)

result = conn.execute("SELECT COUNT(*) FROM sibbr").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Registros carregados na tabela: 10,000,000


In [13]:
conn.execute("DESCRIBE sibbr").df()

,column_name,column_type,null,key,default,extra
0,recordID,VARCHAR,YES,None,None,None
1,DatasetID,VARCHAR,YES,None,None,None
2,DatasetName,VARCHAR,YES,None,None,None
3,institutionID,VARCHAR,YES,None,None,None
4,Institution,VARCHAR,YES,None,None,None
...,...,...,...,...,...,...
98,Type status not recognised,BOOLEAN,YES,None,None,None
99,Unrecognized geodetic datum,BOOLEAN,YES,None,None,None
100,Supplied coordinates are zero,BOOLEAN,YES,None,None,None
101,Zero latitude,BOOLEAN,YES,None,None,None


In [17]:
conn.execute("SELECT * FROM sibbr LIMIT 5").df()

,recordID,DatasetID,DatasetName,institutionID,Institution,collectionID,Collection,license,catalogNumber,TaxonConceptID,...,Supplied country not recognised,Kingdom not recognised,Collection code not recognised,Institution code not recognised,unrecognised Occurrence Status,Type status not recognised,Unrecognized geodetic datum,Supplied coordinates are zero,Zero latitude,Zero longitude
0,f12121a9-b6b9-48f5-9e7d-767796a56576,dr243,ASE - Herbário da Universidade Federal de Sergipe,nan,nan,nan,nan,CC-BY-NC,16840,364349.0,...,True,True,False,False,False,False,False,False,True,False
1,e9574683-06a1-42ab-aead-90f0b5b7ddc6,dr243,ASE - Herbário da Universidade Federal de Sergipe,nan,nan,nan,nan,CC-BY-NC,7096,364359.0,...,True,True,False,False,False,False,False,False,True,False
2,e4b1ba33-cf5c-45b7-9478-8175424d8b5f,dr251,IRATI - Herbário da Universidade Estadual do C...,nan,nan,nan,nan,CC-BY,255,364661.0,...,False,True,True,True,False,False,False,True,True,True
3,e127ad79-8ae1-4dcb-814c-776f7e3ca9c1,dr243,ASE - Herbário da Universidade Federal de Sergipe,nan,nan,nan,nan,CC-BY-NC,3181,364480.0,...,True,True,False,False,False,False,False,False,True,False
4,b61c9b4a-45f7-4550-b2fa-a6981da60dc3,dr251,IRATI - Herbário da Universidade Estadual do C...,nan,nan,nan,nan,CC-BY,564,364529.0,...,False,True,True,True,False,False,False,True,True,True


## Reflora
### Converte json para parquet

In [ ]:
local_file = "./reflora"
dfs = []
for p in pathlib.Path(local_file).rglob("*.json"):
    print(p)
    with open(p, "r", encoding="utf-8") as arquivo:
        data = json.load(arquivo)
    
    if data:
        for d in data:
            aux = d["taxon"]
            if "specie_profile" in d and d["specie_profile"]:
                aux.update({"specie_profile": d["specie_profile"]})
            else:
                aux.update({"specie_profile": {}})
                

            if "reference" in d and d["reference"]:
                aux.update({"reference": d["reference"]})
            else:
                aux.update({"reference": []})
                

            if "distribuition" in d and d["distribuition"]:
                aux.update({"distribuition": d["distribuition"]})
            else:
                aux.update({"distribuition": []})
                
                
            dfs.append(aux)

df = pd.DataFrame(dfs)
df.to_parquet("reflora.parquet", index=False)            

reflora/Rhynchophorum.json
reflora/Peltobryon.json
reflora/Sarcorhachis.json
reflora/Verhuellia.json
reflora/Enckea.json
reflora/Manekia.json
reflora/Troxirum.json
reflora/NA.json
reflora/Artanthe.json
reflora/Piper.json
reflora/Peperomia.json
reflora/Pothomorphe.json
reflora/Acrocarpidium.json
reflora/Micropiper.json
reflora/Carpunya.json
reflora/Ottonia.json


In [23]:
conn.execute("DROP TABLE IF EXISTS reflora")
conn.execute(f"""
        CREATE TABLE reflora AS
        SELECT * FROM read_parquet(reflora.parquet, union_by_name=True)
""")

result = conn.execute("SELECT COUNT(*) FROM reflora").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

Registros carregados na tabela: 1,042


In [24]:
conn.execute("DESCRIBE reflora").df()

,column_name,column_type,null,key,default,extra
0,taxonid,BIGINT,YES,None,None,None
1,scientificname,VARCHAR,YES,None,None,None
2,taxonrank,VARCHAR,YES,None,None,None
3,nomenclaturalstatus,VARCHAR,YES,None,None,None
4,taxonomicstatus,VARCHAR,YES,None,None,None
5,kingdom,VARCHAR,YES,None,None,None
6,phylum,VARCHAR,YES,None,None,None
7,class,VARCHAR,YES,None,None,None
8,order,VARCHAR,YES,None,None,None
9,family,VARCHAR,YES,None,None,None


In [26]:
conn.execute("SELECT * FROM reflora LIMIT 5").df()

,taxonid,scientificname,taxonrank,nomenclaturalstatus,taxonomicstatus,kingdom,phylum,class,order,family,...,originalnameusageid,namepublishedin,namepublishedinyear,references,acceptednameusageid,bibliographiccitation_how_to_cite,modified,specie_profile,reference,distribuition
0,595182,Rhynchophorum obtusifolium (L.) Small,ESPECIE,None,SINONIMO,Plantae,Tracheophyta,Magnoliopsida,Piperales,Piperaceae,...,None,,None,http://reflora.jbrj.gov.br/reflora/listaBrasil...,12681.0,"Guimarães, E.F., Carvalho-Silva, M., Medeiros,...",2020-11-28 11:36:18.643,"{'habitat': None, 'lifeForm': None, 'vegetatio...",[],[]
1,596095,Peltobryon cyrtopodum Miq.,ESPECIE,None,SINONIMO,Plantae,Tracheophyta,Magnoliopsida,Piperales,Piperaceae,...,None,,None,http://reflora.jbrj.gov.br/reflora/listaBrasil...,86594.0,"Guimarães, E.F., Carvalho-Silva, M., Medeiros,...",2019-11-06 14:54:00.248,"{'habitat': None, 'lifeForm': None, 'vegetatio...",[],[]
2,598070,Peltobryon guilleminianum Miq.,ESPECIE,None,SINONIMO,Plantae,Tracheophyta,Magnoliopsida,Piperales,Piperaceae,...,None,,None,http://reflora.jbrj.gov.br/reflora/listaBrasil...,12765.0,"Guimarães, E.F., Carvalho-Silva, M., Medeiros,...",2019-11-06 15:02:15.242,"{'habitat': None, 'lifeForm': None, 'vegetatio...",[],[]
3,594199,Peltobryon exserens Miq.,ESPECIE,None,SINONIMO,Plantae,Tracheophyta,Magnoliopsida,Piperales,Piperaceae,...,None,,None,http://reflora.jbrj.gov.br/reflora/listaBrasil...,12765.0,"Guimarães, E.F., Carvalho-Silva, M., Medeiros,...",2019-11-06 14:55:53.236,"{'habitat': None, 'lifeForm': None, 'vegetatio...",[],[]
4,598970,Peltobryon martianum Miq.,ESPECIE,None,SINONIMO,Plantae,Tracheophyta,Magnoliopsida,Piperales,Piperaceae,...,None,,None,http://reflora.jbrj.gov.br/reflora/listaBrasil...,24212.0,"Guimarães, E.F., Carvalho-Silva, M., Medeiros,...",2020-12-17 12:17:32.656,"{'habitat': None, 'lifeForm': None, 'vegetatio...",[],[]


### IBGE

In [3]:
conn.execute("DROP TABLE IF EXISTS ibge")
conn.execute("""
    CREATE TABLE ibge AS 
    SELECT * FROM read_parquet(ibge.parquet)
""")


result = conn.execute("SELECT COUNT(*) FROM ibge").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

Registros carregados na tabela: 5,571


In [15]:
conn.execute("DESCRIBE ibge").df()

,column_name,column_type,null,key,default,extra
0,id,BIGINT,YES,None,None,None
1,nome_cidade,VARCHAR,YES,None,None,None
2,sigla,VARCHAR,YES,None,None,None
3,nome_estado,VARCHAR,YES,None,None,None


In [4]:
conn.execute("SELECT * FROM ibge LIMIT 5").df()

,id,nome_cidade,sigla,nome_estado
0,1100015,Alta Floresta D'Oeste,RO,Rondônia
1,1100023,Ariquemes,RO,Rondônia
2,1100031,Cabixi,RO,Rondônia
3,1100049,Cacoal,RO,Rondônia
4,1100056,Cerejeiras,RO,Rondônia


## FLORA - Lista de Espécies Ameaçadas - 2021.csv
### Converto para parquet

In [23]:
df = pd.read_csv("flora-ameacada-2021.csv", sep=";", index_col=0, header=0)
df.head()

,Família (FB 2020),Espécie (FB 2020),Nome avaliação anterior,Categoria em 2014,Sugestão de Categoria 2021,EW,CR,"EN,",VU,Mesma Categoria de Ameaça 2014,Aumentou a Categoria de Ameaça,Reduziu a Categoria de Ameaça,Entrou em Categoria de Ameaça,Saíram da Categoria de Ameaça,Saíram da Lista por serem sinonímias taxonômicas
#,,,,,,,,,,,,,,,
1,Acanthaceae,Aphelandra espirito-santensis Profice & Wassh.,Aphelandra espirito-santensis,Em Perigo (EN),Em Perigo (EN),NaN,NaN,X,NaN,X,NaN,NaN,NaN,62 espécies,74 espécies
2,Acanthaceae,Aphelandra margaritae E.Morren,Aphelandra margaritae,Vulnerável (VU),Vulnerável (VU),NaN,NaN,NaN,X,X,NaN,NaN,NaN,NaN,NaN
3,Acanthaceae,Aphelandra maximiliana (Nees) Benth.,Aphelandra maximiliana,Em Perigo (EN),Em Perigo (EN),NaN,NaN,X,NaN,X,NaN,NaN,NaN,NaN,NaN
4,Acanthaceae,Aphelandra rigida Glaz. ex Mildbr.,Aphelandra rigida,NaN,Em Perigo (EN),NaN,NaN,X,NaN,NaN,NaN,NaN,X,NaN,NaN
5,Acanthaceae,Aphelandra stephanophysa Nees,Aphelandra stephanophysa,NaN,Vulnerável (VU),NaN,NaN,NaN,X,NaN,NaN,NaN,X,NaN,NaN


In [24]:
df.to_parquet("flora-ameacada-2021.parquet")
!ls

01download.ipynb	     ibge.parquet   README.md
02load.ipynb		     ibge.py	    reflora.parquet
03cleaning.ipynb	     plants.db	    sibbr
flora-ameacada-2021.csv      plants.db.wal  sibbr_parquet
flora-ameacada-2021.parquet  __pycache__    sibbr.zip


## FLORA - Lista de Espécies Ameaçadas - 2021.csv
### Carrego no duckdb

In [30]:
conn.execute("DROP TABLE IF EXISTS flora_especies_ameacadas")
conn.execute("""
    CREATE TABLE flora_especies_ameacadas AS 
    SELECT * FROM read_parquet('flora-ameacada-2021.parquet')
""")


result = conn.execute("SELECT COUNT(*) FROM flora_especies_ameacadas").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

Registros carregados na tabela: 3,212


In [31]:
conn.execute("DESCRIBE flora_especies_ameacadas").df()

,column_name,column_type,null,key,default,extra
0,Família (FB 2020),VARCHAR,YES,None,None,None
1,Espécie (FB 2020),VARCHAR,YES,None,None,None
2,Nome avaliação anterior,VARCHAR,YES,None,None,None
3,Categoria em 2014,VARCHAR,YES,None,None,None
4,Sugestão de Categoria 2021,VARCHAR,YES,None,None,None
5,EW,DOUBLE,YES,None,None,None
6,CR,VARCHAR,YES,None,None,None
7,"EN,",VARCHAR,YES,None,None,None
8,VU,VARCHAR,YES,None,None,None
9,Mesma Categoria de Ameaça 2014,VARCHAR,YES,None,None,None


In [33]:
conn.execute("SELECT * FROM flora_especies_ameacadas LIMIT 5").df()

,Família (FB 2020),Espécie (FB 2020),Nome avaliação anterior,Categoria em 2014,Sugestão de Categoria 2021,EW,CR,"EN,",VU,Mesma Categoria de Ameaça 2014,Aumentou a Categoria de Ameaça,Reduziu a Categoria de Ameaça,Entrou em Categoria de Ameaça,Saíram da Categoria de Ameaça,Saíram da Lista por serem sinonímias taxonômicas,#
0,Acanthaceae,Aphelandra espirito-santensis Profice & Wassh.,Aphelandra espirito-santensis,Em Perigo (EN),Em Perigo (EN),NaN,None,X,None,X,None,None,None,62 espécies,74 espécies,1
1,Acanthaceae,Aphelandra margaritae E.Morren,Aphelandra margaritae,Vulnerável (VU),Vulnerável (VU),NaN,None,None,X,X,None,None,None,None,None,2
2,Acanthaceae,Aphelandra maximiliana (Nees) Benth.,Aphelandra maximiliana,Em Perigo (EN),Em Perigo (EN),NaN,None,X,None,X,None,None,None,None,None,3
3,Acanthaceae,Aphelandra rigida Glaz. ex Mildbr.,Aphelandra rigida,None,Em Perigo (EN),NaN,None,X,None,None,None,None,X,None,None,4
4,Acanthaceae,Aphelandra stephanophysa Nees,Aphelandra stephanophysa,None,Vulnerável (VU),NaN,None,None,X,None,None,None,X,None,None,5
